In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML
javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}
def toggle_code(state):
    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)
    display(HTML(output))
def button_action(value):
    state = value.new
    toggle_code(state)
    value.owner.description = button_descriptions[state]
state = False
toggle_code(state)
button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")
display(button)

ToggleButton(value=False, description='Show code')

In [3]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)
CSS = """#notebook div.output_subarea {max-width:100%;}""" #changes output_subarea width to 100% (from 100% - 14ex)
HTML('<style>{}</style>'.format(CSS))

import ipywidgets as widgets
from IPython.display import display

from numpy import random
import pandas as pd
import math
from pyecharts.charts import Sunburst, Bar, Line, Scatter, Map, Sankey, Geo, Pie, Grid, Page
from pyecharts import options as opts
from pyecharts.globals import ChartType, SymbolType, ThemeType
from pyecharts.render import make_snapshot
from snapshot_phantomjs import snapshot

# 一、全国各产业创新现状
## 1.1各产业整体数量分布对比（2010-2018年）

In [2]:
industry_df  = pd.read_excel("D:/CASTED/专利/中间表/专利季报-自动出报系统.xlsx", sheet_name = "十年行业占比饼环图")
industry_df.columns = ["二级分类", "三级分类", "专利数"]

class_df = industry_df.groupby(['二级分类']).sum().reset_index().sort_values(by = ["专利数"], ascending = True)
class_df["百分比"] = (class_df["专利数"] / class_df["专利数"].sum() * 100).round(2)
class_df = class_df.sort_values(by = ["百分比"], ascending = False)

##########################################################################################################
## 绘制专利数种类占比饼图
##########################################################################################################
def pie_radius(class_df) -> Pie:
    
    df = class_df
    
    c = (
        Pie(init_opts=opts.InitOpts())
        .add(
            "",
            [list(z) for z in zip(df["二级分类"].tolist(), df["百分比"].tolist())],
            radius=["30%", "45%"],
            is_clockwise = False, 
            center = ["50%", "50%"]
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="各产业整体数量分布对比（2010-2018年）"),
            legend_opts=opts.LegendOpts(
                orient="vertical", pos_top="15%", pos_left="2%"
            ),
        )
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}%"))
        )
             
    return c

c_classPie = pie_radius(class_df)
c_classPie.render_notebook()

## 1.2专利总量及各产业时间变化趋势（2010-2018年）

In [28]:
total_year_df = pd.read_excel("D:/CASTED/专利/中间表/专利季报-自动出报系统.xlsx", sheet_name = "专利总数时间变化趋势（2010-2018年）")
total_year_df.loc["专利总量", :] = total_year_df.sum()
total_year_df.set_index("二级产业", inplace = True)

index = total_year_df.index.tolist()
index[7] = "专利总量"
total_year_df.index = index

##########################################################################################################
## 绘制专利总量逐年变化趋势
##########################################################################################################
def bar_horizon(total_year_df) -> Bar:
    
    df = total_year_df
    
    c = (
        Bar(init_opts=opts.InitOpts(width="960px", height="400px"))
        .add_xaxis(df.columns.tolist())
        .add_yaxis("专利总量", df.loc["专利总量", :].tolist(), category_gap="80%")
        .set_series_opts(label_opts=opts.LabelOpts(position="top"))
        .set_global_opts(
                         #title_opts=opts.TitleOpts(title="专利总数时间变化趋势（2010-2018年）"), 
                         legend_opts = opts.LegendOpts(pos_right = "1%", pos_bottom = "50%", orient = "vertical", is_show = False), 
                         xaxis_opts=opts.AxisOpts(is_show = True))
    )
    return c

c_totalyearBar = bar_horizon(total_year_df)
c_totalyearBar.render_notebook()

In [29]:
rate_year_df = pd.read_excel("D:/CASTED/专利/中间表/专利季报-自动出报系统.xlsx", sheet_name = "各产业逐年增长率时间变化趋势")
rate_year_df.set_index("二级产业", inplace = True)

##########################################################################################################
## 绘制各产业逐年增长率时间变化趋势
##########################################################################################################
def line_base(rate_year_df) -> Line:
    
    df = rate_year_df
    
    c = (
        Line(init_opts=opts.InitOpts(width="960px", height="400px"))
        .add_xaxis([str(x) for x in df.columns.tolist()])
        .add_yaxis("制造业", df.loc['制造业', :].tolist())
        .add_yaxis("居民服务、修理和其他服务业", df.loc['居民服务、修理和其他服务业', :].tolist())
        .add_yaxis("信息传输、软件和信息技术服务业", df.loc['信息传输、软件和信息技术服务业', :].tolist())
        .add_yaxis("建筑业", df.loc['建筑业', :].tolist())
        .add_yaxis("电力、热力、燃气及水生产和供应业", df.loc['电力、热力、燃气及水生产和供应业', :].tolist())
        .add_yaxis("农、林、牧、渔业", df.loc['农、林、牧、渔业', :].tolist())
        .add_yaxis("采矿业", df.loc['采矿业', :].tolist())
        .set_series_opts(label_opts=opts.LabelOpts(is_show = False), 
                         markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(x = "2015")]),
                                                               )
        .set_global_opts(title_opts=opts.TitleOpts(title=""), 
                         yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value}%")), 
                         legend_opts = opts.LegendOpts(pos_right = "1%", pos_top = "1%", orient = "horizontal", is_show = True))
    )
    return c

c_rateLine = line_base(rate_year_df)
c_rateLine.render_notebook()

In [30]:
compete_df = pd.read_excel("D:/CASTED/专利/中间表/专利季报-自动出报系统.xlsx", sheet_name = "2017年各行业竞争对比气泡图")
compete_df.set_index("二级产业", inplace = True)

##########################################################################################################
## 绘制2017年各行业竞争对比气泡图
##########################################################################################################
def scatter_visualmap_color(compete_df) -> Scatter:
    
    df = compete_df
    
    c = (
        Scatter(init_opts=opts.InitOpts(width="960px", height="400px"))
        .add_xaxis([str(x) for x in df["专利数"].tolist()])
        #.add_yaxis("", df["平均增长率"].round(2).tolist())
        .add_yaxis("制造业", df["制造业"].tolist())
        .add_yaxis("居民服务、修理和其他服务业", df["居民服务、修理和其他服务业"].tolist())
        .add_yaxis("信息传输、软件和信息技术服务业", df["信息传输、软件和信息技术服务业"].tolist())
        .add_yaxis("建筑业", df["建筑业"].tolist())
        .add_yaxis("电力、热力、燃气及水生产和供应业", df["电力、热力、燃气及水生产和供应业"].tolist())
        .add_yaxis("农、林、牧、渔业", df["农、林、牧、渔业"].tolist())
        .add_yaxis("采矿业", df["采矿业"].tolist())
        .set_global_opts(
            #title_opts=opts.TitleOpts(title="2017年各行业竞争对比"),
            xaxis_opts=opts.AxisOpts(type_ = "value"),
            yaxis_opts=opts.AxisOpts(is_scale = True),
            visualmap_opts=opts.VisualMapOpts(type_="size", min_ = 18, max_ = 30),
            legend_opts = opts.LegendOpts(is_show = False),
        )
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{a}", position="right"))
    )
    return c

c_bubble = scatter_visualmap_color(compete_df)
c_bubble.render_notebook()

In [6]:
grid = (Grid(init_opts=opts.InitOpts(width="1920px", height="1080px", theme=ThemeType.DARK))
        .add(c_totalyearBar, grid_opts=opts.GridOpts(pos_left="100px", pos_right = "1100px", pos_top = "5%", pos_bottom = "70%"))
        .add(c_rateLine, grid_opts=opts.GridOpts(pos_left="100px", pos_right = "1100px", pos_top = "36%", pos_bottom = "37%"))
        .add(c_bubble, grid_opts=opts.GridOpts(pos_left="100px", pos_right = "1100px", pos_top = "69%", pos_bottom = "5%"))
    )
#make_snapshot(snapshot, grid.render(), "季报全景.png")
# grid.render()

'D:\\CASTED\\专利\\数据准备与模型测试\\render.html'

## 1.3各产业区域分部特点（2010-2018年)

In [7]:
pro_df  = pd.read_excel("D:/CASTED/专利/中间表/1.2-1.3 2010-2018专利数-省-二级产业.xlsx", sheet_name = "省级行政区区域各产业专利规模")

pro_df = pro_df.reset_index()

col_name = pro_df.columns.tolist()
col_name[0] = "省份"
col_name[8] = "总专利数"
pro_df.columns = col_name

##########################################################################################################
## 绘制专利数省份柱状分布地图
##########################################################################################################

def bar_stack(pro_df) -> Bar:
    
    df = pro_df
    
    c = (
        Bar()
        .add_xaxis(df["省份"].tolist())
        .add_yaxis("总专利数", df["总专利数"].tolist())
        .add_yaxis("制造业", df["制造业"].tolist())
        .add_yaxis("居民服务、修理和其他服务业", df["居民服务、修理和其他服务业"].tolist())
        .add_yaxis("信息传输、软件和信息技术服务业", df["信息传输、软件和信息技术服务业"].tolist())
        .add_yaxis("建筑业", df["建筑业"].tolist(), is_selected=False)
        .add_yaxis("电力、热力、燃气及水生产和供应业", df["电力、热力、燃气及水生产和供应业"].tolist(), is_selected=False)
        .add_yaxis("农、林、牧、渔业", df["农、林、牧、渔业"].tolist(), is_selected=False)
        .add_yaxis("采矿业", df["采矿业"].tolist(), is_selected=False)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(title_opts=opts.TitleOpts(title="省级行政区区域各产业专利规模"), 
                         legend_opts = opts.LegendOpts(pos_right = "1%", pos_top = "15%", orient = "vertical"), 
                         datazoom_opts = opts.DataZoomOpts(type_ = "slider", range_start = 0, range_end = 50))
    )
    return c

c_bar_pro = bar_stack(pro_df)
c_bar_pro.render_notebook()

In [8]:
##########################################################################################################
## 绘制专利数省份热力分布地图
##########################################################################################################

def map_visualmap(pro_df) -> Map:
    
    df = pro_df
    
    c = (
        Map()
        .add("总专利数", [list(z) for z in zip(df["省份"].tolist(), df["总专利数"].tolist())], "china")
        .add("制造业", [list(z) for z in zip(df["省份"].tolist(), df["制造业"].tolist())], "china", is_selected=False)
        .add("居民服务、修理和其他服务业", [list(z) for z in zip(df["省份"].tolist(), df["居民服务、修理和其他服务业"].tolist())], "china", is_selected=False)
        .add("信息传输、软件和信息技术服务业", [list(z) for z in zip(df["省份"].tolist(), df["信息传输、软件和信息技术服务业"].tolist())], "china", is_selected=False)
        .add("电力、热力、燃气及水生产和供应业", [list(z) for z in zip(df["省份"].tolist(), df["电力、热力、燃气及水生产和供应业"].tolist())], "china", is_selected=False)
        .add("农、林、牧、渔业", [list(z) for z in zip(df["省份"].tolist(), df["农、林、牧、渔业"].tolist())], "china", is_selected=False)
        .add("采矿业", [list(z) for z in zip(df["省份"].tolist(), df["采矿业"].tolist())], "china", is_selected=False)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="省级行政区区域各产业专利规模"),
            visualmap_opts=opts.VisualMapOpts(max_=df["总专利数"].max().tolist(), is_piecewise=False, orient = "horizontal", pos_bottom = "10%", pos_left = "5%"),
            legend_opts = opts.LegendOpts(pos_right = "1%", pos_bottom = "1%", type_ = "scroll"),
        )
    )
    return c

c_map_GIS = map_visualmap(pro_df)
c_map_GIS.render_notebook()

In [7]:
##########################################################################################################
## 数据清洗，筛选出正确地区并合并重复项，清除不明确地区项
##########################################################################################################
trans_df = pd.read_excel("D:/CASTED/专利/中间表/专利季报-自动出报系统.xlsx", sheet_name = "trans")
def toDistrict(pro):
    if pro in ["黑龙江", "吉林", "辽宁", "内蒙古"]: 
        pro = "东北"
    if pro in ["北京", "天津", "山西", "河北"]:
        pro = "华北"
    if pro in ["上海", "江苏", "浙江", "安徽", "福建", "江西", "山东", "台湾"]:
        pro = "华东"
    if pro in ["河南", "湖北", "湖南", "广西", "广东", "香港", "澳门", "海南"]:
        pro = "华南"
    if pro in ["重庆", "四川", "贵州", "云南", "西藏"]:
        pro = "西南"
    if pro in ["陕西", "甘肃", "青海", "宁夏", "新疆"]:
        pro = "西北"
    
    return pro
trans_district_df = trans_df
trans_district_df[["转入", "转出"]] = trans_df[["转入", "转出"]].applymap(toDistrict)

##########################################################################################################
## 数据清洗，筛选出正确地区并合并重复项，清除不明确地区项
##########################################################################################################
trans_district_df = trans_district_df.groupby(['转出', '转入']).sum().reset_index()

trans_district_df.head(n = 20)

,转出,转入,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,:浙江,华东,0,1,0,0,0,0,0,0,0
1,东,华南,0,1,0,0,0,1,0,0,0
2,东北,东北,1165,2143,4373,2703,2398,2666,3089,4336,4657
3,东北,华东,70,107,287,451,437,305,334,474,913
4,东北,华北,113,131,256,541,540,355,347,372,525
5,东北,华南,26,46,69,76,170,151,273,421,583
6,东北,地址浙江,0,0,0,0,0,0,0,0,0
7,东北,本溪市辽宁,0,0,1,0,0,0,0,0,0
8,东北,西北,0,6,2,13,53,31,22,17,35
9,东北,西南,24,17,10,26,27,33,48,98,72


## 3.1 广东、浙江、江苏专利转移规模最大

In [9]:
##########################################################################################################
## 读取数据、添加列名、将省份转换成区域
##########################################################################################################
trans_df = pd.read_excel("D:/CASTED/专利/中间表/专利季报-自动出报系统.xlsx", sheet_name = "trans")
def toDistrict(pro):
    if pro in ["黑龙江", "吉林", "辽宁", "内蒙古"]: 
        pro = "东北"
    if pro in ["北京", "天津", "山西", "河北"]:
        pro = "华北"
    if pro in ["上海", "江苏", "浙江", "安徽", "福建", "江西", "山东", "台湾"]:
        pro = "华东"
    if pro in ["河南", "湖北", "湖南", "广西", "广东", "香港", "澳门", "海南"]:
        pro = "华南"
    if pro in ["重庆", "四川", "贵州", "云南", "西藏"]:
        pro = "西南"
    if pro in ["陕西", "甘肃", "青海", "宁夏", "新疆"]:
        pro = "西北"
    
    return pro
trans_district_df = trans_df
trans_district_df[["转入", "转出"]] = trans_df[["转入", "转出"]].applymap(toDistrict)

##########################################################################################################
## 数据清洗，筛选出正确地区并合并重复项，清除不明确地区项
##########################################################################################################
trans_district_df = trans_district_df.groupby(['转出', '转入']).sum().reset_index()

def cleanFilter(item):
    district = ["东北", "华北", "华东", "华南", "西南", "西北"]
    if item in district:
        index = True
    else:
        index = False
    return index

trans_district_df = trans_district_df[(trans_district_df["转出"].map(cleanFilter)) & (trans_district_df["转入"].map(cleanFilter))]

##########################################################################################################
## 生成Sankey可用的数据框格式
##########################################################################################################
trans_district_melt = trans_district_df.melt(id_vars = ["转出", "转入"], var_name = "Year", value_name = "Amount")
trans_district_melt["转出-年"] = trans_district_melt["转出"] + (["-"] * len(trans_district_melt)) + [str(i) for i in trans_district_melt["Year"]]
trans_district_melt["转入-年"] = trans_district_melt["转入"] + (["-"] * len(trans_district_melt)) + [str(i+1) for i in trans_district_melt["Year"]]

trans_district_pivot = trans_district_melt.pivot_table(index = ["转出-年"], columns = ["转入-年"], values = ["Amount"])
trans_district_pivot.columns = trans_district_pivot.columns.levels[1].tolist()

##########################################################################################################
## 绘制Sankey图——区域流向图
##########################################################################################################
nodes = []
for item in trans_district_pivot.index:
    nodes.append({"name": item})
for item in trans_district_pivot.columns:
    if item not in trans_district_pivot.index.tolist():
        nodes.append({"name": item})

links = []
for col in trans_district_pivot:
    for index in trans_district_pivot[col].index:
        if not math.isnan(trans_district_pivot[col][index]):
            links.append({"source": index, "target": col, "value": trans_district_pivot[col][index].tolist()})

def sankey_base(node, link) -> Sankey:
    
    nodes = node
    links = link
    
    c = (
        Sankey()
        .add(
            "sankey",
            nodes,
            links,
            linestyle_opt=opts.LineStyleOpts(opacity=0.2, curve=0.5, color="source"),
            label_opts=opts.LabelOpts(position="right"),
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="Sankey-基本示例"))
    )

    return c

c = sankey_base(nodes, links)
c.render_notebook()

In [57]:
##########################################################################################################
## 读取数据、添加列名、将省份转换成区域
##########################################################################################################
trans_df = pd.read_excel("D:/CASTED/专利/中间表/专利季报-自动出报系统.xlsx", sheet_name = "trans")
def toDistrict(pro):
    if pro in ["黑龙江", "吉林", "辽宁", "内蒙古"]: 
        pro = "东北"
    if pro in ["北京", "天津", "山西", "河北"]:
        pro = "华北"
    if pro in ["上海", "江苏", "浙江", "安徽", "福建", "江西", "山东", "台湾"]:
        pro = "华东"
    if pro in ["河南", "湖北", "湖南", "广西", "广东", "香港", "澳门", "海南"]:
        pro = "华南"
    if pro in ["重庆", "四川", "贵州", "云南", "西藏"]:
        pro = "西南"
    if pro in ["陕西", "甘肃", "青海", "宁夏", "新疆"]:
        pro = "西北"
    
    return pro
trans_district_df = trans_df
trans_district_df[["转入", "转出"]] = trans_df[["转入", "转出"]].applymap(toDistrict)
trans_district_df = trans_district_df.loc[:,["转出", "转入", 2018]]

##########################################################################################################
## 数据清洗，筛选出正确地区并合并重复项，清除不明确地区项
##########################################################################################################
trans_district_df = trans_district_df.groupby(['转出', '转入']).sum().reset_index()

def cleanFilter(item):
    district = ["东北", "华北", "华东", "华南", "西南", "西北"]
    if item in district:
        index = True
    else:
        index = False
    return index

trans_district_df = trans_district_df[(trans_district_df["转出"].map(cleanFilter)) & (trans_district_df["转入"].map(cleanFilter))]

##########################################################################################################
## 生成Sankey可用的数据框格式
##########################################################################################################
def trans_out(district):
    district = district + "_转出"
    
    return district

def trans_in(district):
    district = district + "_转入"
    
    return district

trans_district_df["转出"] = trans_district_df["转出"].map(trans_out)
trans_district_df["转入"] = trans_district_df["转入"].map(trans_in)

trans_district_pivot = trans_district_df.pivot_table(index = ["转出"], columns = ["转入"], values = [2018])
trans_district_pivot.columns = trans_district_pivot.columns.levels[1].tolist()

##########################################################################################################
## 绘制Sankey图——区域流向图
##########################################################################################################
nodes = []
for item in trans_district_pivot.index:
    nodes.append({"name": item})
for item in trans_district_pivot.columns:
    if item not in trans_district_pivot.index.tolist():
        nodes.append({"name": item})

links = []
for col in trans_district_pivot:
    for index in trans_district_pivot[col].index:
        if not math.isnan(trans_district_pivot[col][index]):
            links.append({"source": index, "target": col, "value": trans_district_pivot[col][index].tolist()})

def sankey_base(node, link) -> Sankey:
    
    nodes = node
    links = link
    
    c = (
        Sankey()
        .add(
            "sankey",
            nodes,
            links,
            linestyle_opt=opts.LineStyleOpts(opacity=0.2, curve=0.5, color="source"),
            label_opts=opts.LabelOpts(position="right"),
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="2018年专利各区域间转移情况"), 
                         legend_opts = opts.LegendOpts(is_show = False))
    )

    return c

c = sankey_base(nodes, links)
c.render_notebook()

In [11]:
##########################################################################################################
## 读取数据、添加列名、将省份转换成区域
##########################################################################################################
trans_df = pd.read_excel("D:/CASTED/专利/中间表/专利季报-自动出报系统.xlsx", sheet_name = "trans")
trans_df = trans_df.loc[:,["转出", "转入", 2018]]
##########################################################################################################
## 数据清洗，筛选出正确地区并合并重复项，清除不明确地区项
##########################################################################################################
trans_df = trans_df.groupby(['转出', '转入']).sum().reset_index()

def cleanFilter(item):
    province = ["北京", "天津", "河北", "山西", "内蒙古", "辽宁", 
                "吉林", "黑龙江", "上海", "江苏", "浙江", "安徽", 
                "福建", "江西", "山东", "河南", "湖北", "湖南", 
                "广东", "广西", "海南", "重庆", "四川", "贵州", 
                "云南", "西藏", "陕西", "甘肃", "青海", "宁夏", 
                "新疆", "台湾", "香港", "澳门"]
    
    if item in province:
        index = True
    else:
        index = False
    return index

trans_df = trans_df[(trans_df["转出"].map(cleanFilter)) & (trans_df["转入"].map(cleanFilter))]

##########################################################################################################
## 生成Sankey可用的数据框格式
##########################################################################################################
def trans_out(province):
    province = province + "_转出"
    
    return province

def trans_in(province):
    province = province + "_转入"
    
    return province

trans_df["转出"] = trans_df["转出"].map(trans_out)
trans_df["转入"] = trans_df["转入"].map(trans_in)

trans_pivot = trans_df.pivot_table(index = ["转出"], columns = ["转入"], values = [2018])
trans_pivot.columns = trans_pivot.columns.levels[1].tolist()

##########################################################################################################
## 绘制Sankey图——区域流向图
##########################################################################################################
nodes = []
for item in trans_pivot.index:
    nodes.append({"name": item})
for item in trans_pivot.columns:
    if item not in trans_district_pivot.index.tolist():
        nodes.append({"name": item})

links = []
for col in trans_pivot:
    for index in trans_pivot[col].index:
        if not math.isnan(trans_pivot[col][index]):
            links.append({"source": index, "target": col, "value": trans_pivot[col][index].tolist()})

def sankey_base(node, link) -> Sankey:
    
    nodes = node
    links = link
    
    c = (
        Sankey()
        .add(
            "",
            nodes,
            links,
            linestyle_opt=opts.LineStyleOpts(opacity=0.2, curve=0.5, color="source"),
            label_opts=opts.LabelOpts(position="right"),
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="2018年专利各省份间转移情况"))
    )

    return c

c = sankey_base(nodes, links)
c.render_notebook()

In [67]:
##########################################################################################################
## 读取数据、添加列名
##########################################################################################################
trans_net_df = pd.read_excel("D:/CASTED/专利/中间表/专利季报-自动出报系统.xlsx", sheet_name = "各省份转移活跃度、净转出地图")

##########################################################################################################
## 绘制各省份转移活跃度热力分布地图
##########################################################################################################
def map_visualmap(trans_net_df) -> Map:
    
    df = trans_net_df
    
    c = (
        Map()
        .add("转移总量", [list(z) for z in zip(df["省份"].tolist(), df["转移总量"].tolist())], "china")
        #.add("净转出量", [list(z) for z in zip(df["省份"].tolist(), df["净转出量"].tolist())], "china")
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="各省份转移总量热力分布地图"),
            visualmap_opts=opts.VisualMapOpts(
                                              is_piecewise=True, orient = "vertical", pos_bottom = "10%", pos_left = "5%", 
                                              pieces = [{"max": 4999}, {"min": 5000, "max": 9999}, {"min": 10000, "max": 19999}, 
                                                        {"min": 20000, "max": 39999}, {"min": 40000}]),
            legend_opts = opts.LegendOpts(pos_right = "1%", pos_bottom = "1%", type_ = "scroll"),
        )
    )
    return c

c_Trans_GIS = map_visualmap(trans_net_df)
c_Trans_GIS.render_notebook()

In [24]:
##########################################################################################################
## 读取数据、添加列名
##########################################################################################################
trans_net_df = pd.read_excel("D:/CASTED/专利/中间表/专利季报-自动出报系统.xlsx", sheet_name = "各省份转移活跃度、净转出地图")

##########################################################################################################
## 绘制各省份转移活跃度热力分布地图
##########################################################################################################
def map_visualmap(trans_net_df) -> Map:
    
    df = trans_net_df
    
    c = (
        Map()
       # .add("总专利数", [list(z) for z in zip(df["省份"].tolist(), df["转移总量"].tolist())], "china")
        .add("净转出量", [list(z) for z in zip(df["省份"].tolist(), df["净转出量"].tolist())], "china")
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="各省份净转出热力分布地图"),
            visualmap_opts=opts.VisualMapOpts(
                                              is_piecewise=True, orient = "vertical", pos_bottom = "10%", pos_left = "5%", 
                                              pieces = [{"max": -3000}, {"min": -2999, "max": -1000}, 
                                                        {"min": -999, "max": 0}, {"min": 1, "max": 1000}, 
                                                        {"min": 1001, "max": 4000}, {"min": 4001}]),
            legend_opts = opts.LegendOpts(pos_right = "1%", pos_bottom = "1%", type_ = "scroll"),
        )
    )
    return c

c_netTrans_GIS = map_visualmap(trans_net_df)
c_netTrans_GIS.render_notebook()

In [55]:
##########################################################################################################
## 绘制各省份净转出分布
##########################################################################################################
def bar_horizon(trans_net_df) -> Bar:
    
    df = trans_net_df
    df = df.sort_values(by = "净转出量", ascending = False)
    
    c = (
        Bar(init_opts=opts.InitOpts(width="960px", height="500px"))
        .add_xaxis(df["省份"].tolist())
        .add_yaxis("转移总量", df["专利总量"].tolist(), category_gap="80%", is_selected = False)
        .add_yaxis("净转出量", df["净转出量"].tolist(), category_gap="80%")
        #.reversal_axis()
        .set_series_opts(label_opts=opts.LabelOpts(position="right", is_show = False))
        .set_global_opts(
                         #title_opts=opts.TitleOpts(title="专利总数时间变化趋势（2010-2018年）"), 
                         legend_opts = opts.LegendOpts(pos_right = "1%", pos_bottom = "50%", orient = "vertical", is_show = True), 
                         xaxis_opts=opts.AxisOpts(is_show = True, axislabel_opts=opts.LabelOpts(rotate=45)), 
                         yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True)), 
                         datazoom_opts = opts.DataZoomOpts(type_ = "slider", range_start = 0, range_end = 50))
    )
    return c

c_transNet_Bar = bar_horizon(trans_net_df)
c_transNet_Bar.render_notebook()

## 3.2 北京辐射带动能力最为强劲

In [83]:
##########################################################################################################
## 读取数据、添加列名
##########################################################################################################
trans_df = pd.read_excel("D:/CASTED/专利/中间表/专利季报-自动出报系统.xlsx", sheet_name = "trans")
trans_df.columns = ["转出", "转入", "2010年", "2011年", "2012年", "2013年", "2014年", "2015年", "2016年", "2017年", "2018年"]

##########################################################################################################
## 数据清洗，筛选出正确地区并合并重复项，清除不明确地区项
##########################################################################################################
trans_df = trans_df.groupby(['转出', '转入']).sum().reset_index()

def cleanFilter(item):
    province = ["北京", "天津", "河北", "山西", "内蒙古", "辽宁", 
                "吉林", "黑龙江", "上海", "江苏", "浙江", "安徽", 
                "福建", "江西", "山东", "河南", "湖北", "湖南", 
                "广东", "广西", "海南", "重庆", "四川", "贵州", 
                "云南", "西藏", "陕西", "甘肃", "青海", "宁夏", 
                "新疆", "台湾", "香港", "澳门"]
    
    if item in province:
        index = True
    else:
        index = False
    return index

trans_df = trans_df[(trans_df["转出"].map(cleanFilter)) & (trans_df["转入"].map(cleanFilter))]

##########################################################################################################
## 筛选出指定地区年份的转入、转出数据
##########################################################################################################
def trans_city(trans_df, city = "北京", year = "2018年", local_included = True):
    trans_out = trans_df[trans_df["转出"] == city][["转出", "转入", year]]
    trans_out_name = trans_out.columns.tolist()
    trans_out_name[2] = "专利转移数量"
    trans_out.columns = trans_out_name
    trans_in = trans_df[trans_df["转入"] == city][["转出", "转入", year]]
    trans_in_name = trans_in.columns.tolist()
    trans_in_name[2] = "专利转移数量"
    trans_in.columns = trans_in_name
    
    if not local_included:
        trans_out = trans_out[trans_out["转入"] != city]
        trans_in = trans_in[trans_in["转出"] != city]
    
    return trans_out, trans_in

trans_out_北京, trans_in_北京 = trans_city(trans_df, "北京", "2018年", local_included = True)

##########################################################################################################
## 绘制专利省份转移热力分布地图
##########################################################################################################
def geo_lines(trans_out, trans_in) -> Geo:
    
    df_out = trans_out
    df_in = trans_in
    
    c = (
        Geo()
        .add_schema(maptype="china")
        .add(
            "转出",
            list(zip(df_out["转入"].tolist(), df_out["专利转移数量"].tolist())),
            type_=ChartType.HEATMAP,
            #color="white",
        )
        .add(
            "转出",
            list(zip(df_out["转出"].tolist(), df_out["转入"].tolist())),
            type_=ChartType.LINES,
            #effect_opts=opts.EffectOpts(
            #    symbol=SymbolType.ARROW, symbol_size=6, color="blue"
            #),
            linestyle_opts=opts.LineStyleOpts(curve=0.2),
        )
        .add(
            "转入",
            list(zip(df_in["转出"].tolist(), df_in["专利转移数量"].tolist())),
            type_=ChartType.HEATMAP,
            #color="white",
        )
        .add(
            "转入",
            list(zip(df_in["转出"].tolist(), df_in["转入"].tolist())),
            type_=ChartType.LINES,
            #effect_opts=opts.EffectOpts(
            #    symbol=SymbolType.ARROW, symbol_size=6, color="blue"
            #),
            linestyle_opts=opts.LineStyleOpts(curve=0.2),
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            visualmap_opts=opts.VisualMapOpts(orient = "horizontal", pos_bottom = "10%", pos_left = "10%"),
            title_opts=opts.TitleOpts(title="北京专利转移辐射图"), 
        )
    )
    return c

c_trans_北京 = geo_lines(trans_out_北京, trans_in_北京)
c_trans_北京.render_notebook()

In [6]:
# 用户输入需要查询的省份
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt

##########################################################################################################
## 筛选出指定地区年份的转入、转出数据
##########################################################################################################
@widgets.interact_manual(
    pro = ["北京", "天津", "河北", "山西", "内蒙古", "辽宁", 
           "吉林", "黑龙江", "上海", "江苏", "浙江", "安徽", 
           "福建", "江西", "山东", "河南", "湖北", "湖南", 
           "广东", "广西", "海南", "重庆", "四川", "贵州", 
           "云南", "西藏", "陕西", "甘肃", "青海", "宁夏", 
           "新疆", "台湾", "香港", "澳门"], 
    year = ["2010年", "2011年", "2012年", "2013年", "2014年", "2015年", "2016年", "2017年", "2018年"])


def geo_lines(pro = "北京", year = "2018年", local_included = True) -> Geo:
    
    def trans_city(pro, year, local_included):
        trans_df = pd.read_excel("D:/CASTED/专利/中间表/专利季报-自动出报系统.xlsx", sheet_name = "trans")
        trans_df.columns = ["转出", "转入", "2010年", "2011年", "2012年", "2013年", "2014年", "2015年", "2016年", "2017年", "2018年"]
    
        trans_df = trans_df.groupby(['转出', '转入']).sum().reset_index()
        def cleanFilter(item):
            province = ["北京", "天津", "河北", "山西", "内蒙古", "辽宁", 
                        "吉林", "黑龙江", "上海", "江苏", "浙江", "安徽", 
                        "福建", "江西", "山东", "河南", "湖北", "湖南", 
                        "广东", "广西", "海南", "重庆", "四川", "贵州", 
                        "云南", "西藏", "陕西", "甘肃", "青海", "宁夏", 
                        "新疆", "台湾", "香港", "澳门"]
    
            if item in province:
                index = True
            else:
                index = False
            return index

        trans_df = trans_df[(trans_df["转出"].map(cleanFilter)) & (trans_df["转入"].map(cleanFilter))]
    
    
        trans_out = trans_df[trans_df["转出"] == pro][["转出", "转入", year]]
        trans_out_name = trans_out.columns.tolist()
        trans_out_name[2] = "专利转移数量"
        trans_out.columns = trans_out_name
        trans_in = trans_df[trans_df["转入"] == pro][["转出", "转入", year]]
        trans_in_name = trans_in.columns.tolist()
        trans_in_name[2] = "专利转移数量"
        trans_in.columns = trans_in_name
    
        if not local_included:
            trans_out = trans_out[trans_out["转入"] != pro]
            trans_in = trans_in[trans_in["转出"] != pro]

        return trans_out, trans_in
    
    
    df_out, df_in = trans_city(pro, year, local_included)
    
    c = (
        Geo()
        .add_schema(maptype="china")
        .add(
            "转出",
            list(zip(df_out["转入"].tolist(), df_out["专利转移数量"].tolist())),
            type_=ChartType.HEATMAP,
            #color="white",
        )
        .add(
            "转出",
            list(zip(df_out["转出"].tolist(), df_out["转入"].tolist())),
            type_=ChartType.LINES,
            #effect_opts=opts.EffectOpts(
            #    symbol=SymbolType.ARROW, symbol_size=6, color="blue"
            #),
            linestyle_opts=opts.LineStyleOpts(curve=0.2),
        )
        .add(
            "转入",
            list(zip(df_in["转出"].tolist(), df_in["专利转移数量"].tolist())),
            type_=ChartType.HEATMAP,
            #color="white",
            is_selected = False, 
        )
        .add(
            "转入",
            list(zip(df_in["转出"].tolist(), df_in["转入"].tolist())),
            type_=ChartType.LINES,
            #effect_opts=opts.EffectOpts(
            #    symbol=SymbolType.ARROW, symbol_size=6, color="blue"
            #),
            linestyle_opts=opts.LineStyleOpts(curve=0.2),
            is_selected = False, 
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            visualmap_opts=opts.VisualMapOpts(orient = "horizontal", pos_bottom = "10%", pos_left = "10%"),
            title_opts=opts.TitleOpts(title="专利转移辐射图"), 
        )
    )
    return c.render_notebook()



interactive(children=(Dropdown(description='pro', options=('北京', '天津', '河北', '山西', '内蒙古', '辽宁', '吉林', '黑龙江', '…

In [13]:
##########################################################################################################
## 绘制北京卖出专利数各省份柱状分布图
##########################################################################################################
def bar_stack(trans_out, trans_in) -> Bar:
    
    df = pd.merge(trans_out, trans_in, how = 'inner', left_on = "转入", right_on = "转出", 
                  suffixes=('_由', '_到'))[["转入_由", "专利转移数量_由", "专利转移数量_到"]]
    df.columns = ["省份", "转出", "转入"]
    df = df.sort_values(by = ["转出"], ascending = False)
    
    c = (
        Bar()
        .add_xaxis(df["省份"].tolist())
        .add_yaxis("转出", df["转出"].tolist())
        .add_yaxis("转入", df["转入"].tolist(), is_selected=False, category_gap="80%")
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(title_opts=opts.TitleOpts(title="北京市卖出专利流向省市规模对比（2018年）"), 
                         legend_opts = opts.LegendOpts(pos_right = "1%", pos_top = "15%", orient = "vertical"), 
                         #datazoom_opts = opts.DataZoomOpts(type_ = "inside", range_start = 0, range_end = 40), 
                        )
    )
    return c

c_bar_city = bar_stack(trans_out_北京, trans_in_北京)
c_bar_city.render_notebook()

In [14]:
##########################################################################################################
## 绘制北京卖出专利数各省份分布饼图
##########################################################################################################
def pie_base(trans_out, trans_in) -> Pie:
    
    df = pd.merge(trans_out, trans_in, how = 'inner', left_on = "转入", right_on = "转出", 
                  suffixes=('_由', '_到'))[["转入_由", "专利转移数量_由", "专利转移数量_到"]]
    df.columns = ["省份", "转出", "转入"]
    df = df.sort_values(by = ["转出"], ascending = False)
    
    c = (
        Pie(init_opts=opts.InitOpts(width="800px", height="800px"))
        .add("", [list(z) for z in zip(df["省份"].tolist(), df["转出"].tolist())], radius=["40%", "75%"])
        .set_global_opts(title_opts=opts.TitleOpts(title="Pie-基本示例"), 
                         legend_opts=opts.LegendOpts(pos_bottom="1%"))
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    )
    return c

c_pie_city = pie_base(trans_out_北京, trans_in_北京)
c_pie_city.render_notebook()